# Exploring European Soccer Database

### Objective:
- Utilize SQL queries to systematically analyze the European Soccer Database.
- Employ rigorous exploration and interpretation methods throughout the project.
- Identify patterns, trends, and performance metrics within the dataset.
- Provide a comprehensive understanding of team dynamics in European football.
- Contribute insights that hold value within the broader context of European soccer.
- Deliver findings that bring meaningful insights and interesting facts to light.
- Leverage SQL expertise as the primary tool for drawing valuable information from the database.
- Enhance decision-making processes within the realm of European football through the gathered insights.

Here you can access every table used in the course. To access each table, you will need to specify the `soccer` schema in your queries (e.g., `soccer.match` for the `match` table, and `soccer.league` for the `league` table).



**Question** 1

Do the number of soccer matches played in a given European country differ across seasons?

In [1]:
SELECT 
	c.name AS country,
    -- Count matches in each of the 3 seasons
	COUNT(CASE WHEN m.season = '2012/2013' THEN m.id END) AS matches_2012_2013,
	COUNT(CASE WHEN m.season = '2013/2014' THEN m.id END) AS matches_2013_2014,
	COUNT(CASE WHEN m.season = '2014/2015' THEN m.id END) AS matches_2014_2015
FROM soccer.country AS c
LEFT JOIN soccer.match AS m
ON c.id = m.country_id
GROUP BY country;

,country,matches_2012_2013,matches_2013_2014,matches_2014_2015
0,Portugal,240,240,306
1,France,380,380,380
2,Scotland,228,228,228
3,Netherlands,306,306,306
4,Spain,380,380,380
5,Belgium,240,12,240
6,Italy,380,380,379
7,Germany,306,306,306
8,England,380,380,380
9,Switzerland,180,180,180


The number of matches played in each season seems relatively consistent across countries.

**Question** 2

Determine the total number of matches won by the home team in each country during the 2012/2013, 2013/2014, and 2014/2015 seasons.

In [2]:
SELECT 
	c.name AS country,
    -- Sum the total records in each season where the home team won
	SUM(CASE WHEN m.season = '2012/2013' AND m.home_goal > m.away_goal 
        THEN 1 ELSE 0 END) AS matches_2012_2013,
 	SUM(CASE WHEN m.season = '2013/2014' AND m.home_goal > m.away_goal 
        THEN 1 ELSE 0 END) AS matches_2013_2014,
	SUM(CASE WHEN m.season = '2014/2015' AND m.home_goal > m.away_goal 
        THEN 1 ELSE 0 END) AS matches_2014_2015
FROM soccer.country AS c
LEFT JOIN soccer.match AS m
ON c.id = m.country_id
-- Group by country name alias
GROUP BY country;

,country,matches_2012_2013,matches_2013_2014,matches_2014_2015
0,Portugal,103,108,137
1,France,170,168,181
2,Scotland,89,102,102
3,Netherlands,137,144,138
4,Spain,189,179,171
5,Belgium,102,6,106
6,Italy,177,181,152
7,Germany,130,145,145
8,England,166,179,172
9,Switzerland,84,82,76


**Question** 3

Display the number of matches played in Great Britain versus elsewhere in the world.
- "England", "Scotland", and "Wales" should be categorized as "Great Britain"
- All other leagues will need to be categorized as "World".

In [3]:
WITH match_location AS (SELECT
	m.id,
	m.country_id,
	c.name AS country,
	CASE WHEN c.name = 'England' THEN 'Great Britan'
		WHEN c.name = 'Scotland' THEN 'Great Britan'
		WHEN c.name = 'Wales' THEN 'Great Britan'
		ELSE 'World' END AS location
FROM soccer.match AS m
LEFT JOIN soccer.country AS c
ON m.country_id = c.id)

SELECT location, COUNT(id)
FROM match_location
GROUP BY location;

,location,count
0,World,10405
1,Great Britan,2432


**Question** 4

Examine the number of wins, losses, and ties in each country.

In [4]:
-- Will have to create 3 CASE statements to COUNT the total number of home team wins, away team wins, and ties.

SELECT 
    c.name AS country,
    -- Count the home wins, away wins, and ties in each country
	COUNT(CASE WHEN m.home_goal > m.away_goal THEN m.id 
        END) AS home_team_wins,
	COUNT(CASE WHEN m.home_goal < m.away_goal THEN m.id 
        END) AS away_team_wins,
	COUNT(CASE WHEN m.home_goal = m.away_goal THEN m.id 
        END) AS ties
FROM soccer.country AS c
LEFT JOIN soccer.match AS m
ON c.id = m.country_id
GROUP BY country;

,country,home_team_wins,away_team_wins,ties
0,Portugal,463,304,259
1,France,698,410,412
2,Scotland,383,305,224
3,Netherlands,574,352,298
4,Spain,727,438,355
5,Belgium,333,213,186
6,Italy,675,415,407
7,Germany,559,362,303
8,England,688,460,372
9,Switzerland,312,216,174


**Question** 5

Continuing on the above question, examine the percentage of of ties in each country for seasons 2013/2014 and 2014/2015.

In [5]:
SELECT 
	c.name AS country,
    -- Rounding the percentage of tied games to 2 decimal points
	ROUND(AVG(CASE WHEN m.season='2013/2014' AND m.home_goal = m.away_goal THEN 1
			 WHEN m.season='2013/2014' AND m.home_goal != m.away_goal THEN 0
			 END), 2) AS pct_ties_2013_2014,
	ROUND(AVG(CASE WHEN m.season='2014/2015' AND m.home_goal = m.away_goal THEN 1
			 WHEN m.season='2014/2015' AND m.home_goal != m.away_goal THEN 0
			 END), 2) AS pct_ties_2014_2015
FROM soccer.country AS c
LEFT JOIN soccer.match AS m
ON c.id = m.country_id
GROUP BY country;

,country,pct_ties_2013_2014,pct_ties_2014_2015
0,Portugal,0.25,0.28
1,France,0.28,0.23
2,Scotland,0.22,0.19
3,Netherlands,0.27,0.24
4,Spain,0.23,0.24
5,Belgium,0.17,0.25
6,Italy,0.24,0.32
7,Germany,0.21,0.27
8,England,0.21,0.24
9,Switzerland,0.23,0.27


**Question** 6

Generate a list of matches where the total goals scored (for both teams in total) is more than 3 times the average for games in the matches_2013_2014 table.

In [6]:
SELECT
    date,
	home_goal,
	away_goal
FROM soccer.match
WHERE 
	season = '2013/2014'
	AND (home_goal + away_goal) > 
       (SELECT 3 * AVG(home_goal + away_goal)
        FROM soccer.match
		WHERE season = '2013/2014'); 

,date,home_goal,away_goal
0,2013-12-14 00:00:00+00:00,6,3
1,2014-03-22 00:00:00+00:00,3,6
2,2013-10-30 00:00:00+00:00,7,3


**Question** 7

Let's evaluate the count of matches with 10 or more goals in each country, and the corresponding country names?

In [7]:
SELECT
    c.name AS country_name,
    COUNT(sub.id) AS matches
FROM soccer.country AS c
INNER JOIN (SELECT country_id, id 
           FROM soccer.match
           -- Filter the subquery by matches with 10+ goals
           WHERE (home_goal + away_goal) >= 10) AS sub
ON c.id = sub.country_id
GROUP BY country_name;

,country_name,matches
0,Netherlands,1
1,Spain,4
2,Germany,1
3,England,3


In [8]:
-- Alternative approach without using join
-- Just for testing knowledge :)

SELECT
    c.name AS country_name,
    COUNT(sub.id) AS matches
FROM soccer.country AS c,
	(SELECT country_id, id 
	FROM soccer.match
	WHERE (home_goal + away_goal) >= 10) AS sub
WHERE c.id = sub.country_id
GROUP BY country_name;

,country_name,matches
0,Netherlands,1
1,Spain,4
2,Germany,1
3,England,3


**Question** 8

In the previous question, we found that England, Netherlands, Germany and Spain were the only countries that had matches in the database where 10 or more goals were scored overall. 

Now, let's find out some more details about those matches -- when they were played, during which seasons, and how many of the goals were home versus away goals.

In [9]:
SELECT
    country,
    date,
    home_goal,
    away_goal
FROM 
	(SELECT c.name AS country, 
     	    m.date, 
     		m.home_goal, 
     		m.away_goal,
           (m.home_goal + m.away_goal) AS total_goals
    FROM soccer.match AS m
    LEFT JOIN soccer.country AS c
    ON m.country_id = c.id) AS subq
-- Filter by total goals scored in the main query
WHERE total_goals >= 10;

,country,date,home_goal,away_goal
0,England,2011-08-28 00:00:00+00:00,8,2
1,England,2012-12-29 00:00:00+00:00,7,3
2,England,2013-05-19 00:00:00+00:00,5,5
3,Germany,2013-03-30 00:00:00+00:00,9,2
4,Netherlands,2011-11-06 00:00:00+00:00,6,4
5,Spain,2013-10-30 00:00:00+00:00,7,3
6,Spain,2015-04-05 00:00:00+00:00,9,1
7,Spain,2015-05-23 00:00:00+00:00,7,3
8,Spain,2014-09-20 00:00:00+00:00,2,8


**Question** 9

Generated a list of leagues and the number of matches in each league with more than 10 total goals.

In [10]:
WITH match_list AS (
    SELECT 
  		country_id, 
  		id
    FROM soccer.match
    WHERE (home_goal + away_goal) >= 10)

SELECT
    l.name AS league,
    COUNT(match_list.id) AS matches
FROM soccer.league AS l
LEFT JOIN match_list ON l.id = match_list.country_id
GROUP BY l.name
ORDER BY matches DESC;

,league,matches
0,Spain LIGA BBVA,4
1,England Premier League,3
2,Netherlands Eredivisie,1
3,Germany 1. Bundesliga,1
4,Italy Serie A,0
5,Portugal Liga ZON Sagres,0
6,Switzerland Super League,0
7,Belgium Jupiler League,0
8,Poland Ekstraklasa,0
9,Scotland Premier League,0


**Question** 10

In the previous question, we identified the number of matches in each league with more than 10 total goals in a match. Now, let's see the details of those matches.

In [11]:
WITH match_list AS (
    SELECT 
  		l.name AS league, 
     	m.date, 
  		m.home_goal, 
  		m.away_goal,
        (m.home_goal + m.away_goal) AS total_goals
    FROM soccer.match AS m
    LEFT JOIN soccer.league AS l
	ON m.country_id = l.id)

SELECT league, date, home_goal, away_goal
FROM match_list
WHERE total_goals >= 10;

,league,date,home_goal,away_goal
0,England Premier League,2011-08-28 00:00:00+00:00,8,2
1,England Premier League,2012-12-29 00:00:00+00:00,7,3
2,England Premier League,2013-05-19 00:00:00+00:00,5,5
3,Germany 1. Bundesliga,2013-03-30 00:00:00+00:00,9,2
4,Netherlands Eredivisie,2011-11-06 00:00:00+00:00,6,4
5,Spain LIGA BBVA,2013-10-30 00:00:00+00:00,7,3
6,Spain LIGA BBVA,2015-04-05 00:00:00+00:00,9,1
7,Spain LIGA BBVA,2015-05-23 00:00:00+00:00,7,3
8,Spain LIGA BBVA,2014-09-20 00:00:00+00:00,2,8


**Question** 11

Calculate the average number of goals scored overall and then include the aggregate value in each row.

In [12]:
SELECT
	m.id,
	c.name AS country,
	season,
	home_goal,
	away_goal,
	AVG(home_goal + away_goal) OVER() AS overall_avg
FROM soccer.match as m
LEFT JOIN soccer.country as c
ON m.country_id = c.id;

,id,country,season,home_goal,away_goal,overall_avg
0,757,Belgium,2011/2012,2,1,2.732103
1,758,Belgium,2011/2012,1,1,2.732103
2,759,Belgium,2011/2012,3,1,2.732103
3,760,Belgium,2011/2012,0,1,2.732103
4,761,Belgium,2011/2012,0,0,2.732103
...,...,...,...,...,...,...
12832,25795,Switzerland,2014/2015,3,1,2.732103
12833,25796,Switzerland,2014/2015,3,2,2.732103
12834,25797,Switzerland,2014/2015,3,0,2.732103
12835,25798,Switzerland,2014/2015,0,0,2.732103


**Question** 12

Rank the leagues according to the outcome of the fact - matches of which leagues, on average, scores the most goals in a match. Compare it with total average goal score.

In [13]:
-- My initial appraoch. However, if we continue along this path, we have to use the below query as a subquery to get the desired result.
-- Instead, we will proceed with a different approach.
-- Just documenting this query for reference purpose.

SELECT
    DISTINCT(l.name) AS league,
    AVG(home_goal + away_goal) OVER (PARTITION BY l.id) AS avg_goals
FROM soccer.league AS l
LEFT JOIN soccer.match AS m
ON l.id = m.country_id
ORDER BY avg_goals DESC;

,league,avg_goals
0,Netherlands Eredivisie,3.170752
1,Germany 1. Bundesliga,2.927288
2,Belgium Jupiler League,2.857923
3,Spain LIGA BBVA,2.759868
4,Switzerland Super League,2.740741
5,England Premier League,2.734211
6,Scotland Premier League,2.672149
7,Italy Serie A,2.659319
8,Portugal Liga ZON Sagres,2.566277
9,France Ligue 1,2.501974


In [14]:
SELECT
	l.name AS league,
	AVG(home_goal + away_goal) AS avg_goals,
	AVG(AVG(home_goal + away_goal)) OVER() AS overall_avg,
	RANK() OVER(ORDER BY AVG(home_goal + away_goal) DESC) AS league_rank
FROM soccer.league AS l
LEFT JOIN soccer.match AS m
ON l.id = m.country_id
GROUP BY l.name
ORDER BY league_rank;

,league,avg_goals,overall_avg,league_rank
0,Netherlands Eredivisie,3.170752,2.734269,1
1,Germany 1. Bundesliga,2.927288,2.734269,2
2,Belgium Jupiler League,2.857923,2.734269,3
3,Spain LIGA BBVA,2.759868,2.734269,4
4,Switzerland Super League,2.740741,2.734269,5
5,England Premier League,2.734211,2.734269,6
6,Scotland Premier League,2.672149,2.734269,7
7,Italy Serie A,2.659319,2.734269,8
8,Portugal Liga ZON Sagres,2.566277,2.734269,9
9,France Ligue 1,2.501974,2.734269,10


**Question** 13

Context : In soccer leagues, games are played at different stages. Winning teams progress from one stage to the next, until they reach the final stage. In each stage, the stakes become higher than the previous one. The match table includes data about the different stages that each match took place in. 

Extract data examining the average goals scored in each stage of a match. Does the average number of goals scored change as the stakes get higher from one stage to the next? Let's examine the 2012/2013 season.

In [15]:
SELECT 
	m.stage,
    ROUND(AVG(m.home_goal + m.away_goal),2) AS avg_goals,
    -- Select the average overall goals for the 2012/2013 season
    ROUND((SELECT AVG(home_goal + away_goal) 
           FROM soccer.match 
           WHERE season = '2012/2013'),2) AS overall
FROM soccer.match AS m
-- Filter for the 2012/2013 season
WHERE season = '2012/2013'
GROUP BY m.stage
ORDER BY avg_goals DESC;

,stage,avg_goals,overall
0,12,3.23,2.77
1,38,3.17,2.77
2,33,3.10,2.77
3,8,3.09,2.77
4,31,3.06,2.77
5,23,3.01,2.77
6,20,2.96,2.77
7,10,2.96,2.77
8,11,2.92,2.77
9,21,2.90,2.77


**Question** 14

Now, filter out the stages where the average goals scored in each stage, exceeds the overall average average number of goals in the season.

In [16]:
SELECT 
	s.stage,
    ROUND(s.avg_goals,2) AS avg_goal,
    -- Select the overall average for 2012/2013
    (	SELECT AVG(home_goal + away_goal)
		FROM soccer.match
		WHERE season = '2012/2013'
	) AS overall_avg
FROM 
	(SELECT
		 stage,
         AVG(home_goal + away_goal) AS avg_goals
	 FROM soccer.match
	 WHERE season = '2012/2013'
	 GROUP BY stage) AS s
WHERE 
	s.avg_goals > (SELECT AVG(home_goal + away_goal)
                    FROM soccer.match
					WHERE season = '2012/2013');

,stage,avg_goal,overall_avg
0,4,2.80,2.772699
1,10,2.96,2.772699
2,38,3.17,2.772699
3,6,2.78,2.772699
4,12,3.23,2.772699
5,36,2.90,2.772699
6,31,3.06,2.772699
7,30,2.87,2.772699
8,21,2.90,2.772699
9,3,2.83,2.772699


**Question** 15

Examine matches with scores that are extreme outliers for each country. Extreme outliers are scores that are above 3 times the average score.

In [17]:
SELECT
	main.country_id,
	main.date,
	main.home_goal, 
	main.away_goal
FROM soccer.match AS main
WHERE 
	-- Filter the main query by the subquery
	(home_goal + away_goal) > 
        (SELECT AVG((sub.home_goal + sub.away_goal) * 3)
         FROM soccer.match AS sub
         -- Join the main query to the subquery in WHERE
         WHERE main.country_id = sub.country_id);

,country_id,date,home_goal,away_goal
0,1,2011-10-29 00:00:00+00:00,4,5
1,1729,2011-08-28 00:00:00+00:00,8,2
2,1729,2012-12-29 00:00:00+00:00,7,3
3,1729,2013-05-19 00:00:00+00:00,5,5
4,1729,2013-12-14 00:00:00+00:00,6,3
5,1729,2014-03-22 00:00:00+00:00,3,6
6,1729,2014-08-30 00:00:00+00:00,3,6
7,4769,2011-10-15 00:00:00+00:00,5,3
8,4769,2011-12-21 00:00:00+00:00,4,4
9,4769,2012-02-12 00:00:00+00:00,4,5


In [18]:
-- Another approach for practice.

SELECT
	m.country_id, m.date, m.home_goal, m.away_goal
FROM soccer.match AS m
LEFT JOIN (	SELECT
				country_id,
				3 * AVG(home_goal + away_goal) AS country_extreme
			FROM soccer.match
			GROUP BY country_id) AS sub
ON m.country_id = sub.country_id
WHERE (m.home_goal + m.away_goal) > sub.country_extreme;

,country_id,date,home_goal,away_goal
0,1,2011-10-29 00:00:00+00:00,4,5
1,1729,2011-08-28 00:00:00+00:00,8,2
2,1729,2012-12-29 00:00:00+00:00,7,3
3,1729,2013-05-19 00:00:00+00:00,5,5
4,1729,2013-12-14 00:00:00+00:00,6,3
5,1729,2014-03-22 00:00:00+00:00,3,6
6,1729,2014-08-30 00:00:00+00:00,3,6
7,4769,2011-10-15 00:00:00+00:00,5,3
8,4769,2011-12-21 00:00:00+00:00,4,4
9,4769,2012-02-12 00:00:00+00:00,4,5


**Question** 16

For the previous question, we generated a list of matches with extremely high scores for each country. Now, let's examine the highest scoring match for each country, in each season.

In [19]:
SELECT
	main.country_id,
    main.date,
    main.home_goal,
    main.away_goal
FROM soccer.match AS main
WHERE 
	-- Filter for matches with the highest number of goals scored
	(home_goal + away_goal) = 
        (SELECT MAX(sub.home_goal + sub.away_goal)
         FROM soccer.match AS sub
         WHERE main.country_id = sub.country_id
               AND main.season = sub.season);

,country_id,date,home_goal,away_goal
0,1,2011-10-29 00:00:00+00:00,4,5
1,1,2012-11-17 00:00:00+00:00,2,6
2,1,2012-12-09 00:00:00+00:00,1,7
3,1,2013-01-19 00:00:00+00:00,2,6
4,1,2012-08-19 00:00:00+00:00,2,6
...,...,...,...,...
73,24558,2012-09-30 00:00:00+00:00,6,2
74,24558,2014-02-16 00:00:00+00:00,5,3
75,24558,2015-04-30 00:00:00+00:00,6,2
76,24558,2015-05-03 00:00:00+00:00,2,6


In [20]:
-- Another approach for practice :)

SELECT
	s.country_id,
	s.season,
	date,
	home_goal,
	away_goal
FROM soccer.match AS s,
	(SELECT
		country_id,
		season,
		MAX(home_goal + away_goal) as max_goals
	FROM soccer.match
	GROUP BY country_id, season) AS sub
WHERE (home_goal + away_goal) = sub.max_goals
	AND s.country_id = sub.country_id
	AND s.season = sub.season;

,country_id,season,date,home_goal,away_goal
0,1,2011/2012,2011-10-29 00:00:00+00:00,4,5
1,1,2012/2013,2012-11-17 00:00:00+00:00,2,6
2,1,2012/2013,2012-12-09 00:00:00+00:00,1,7
3,1,2012/2013,2013-01-19 00:00:00+00:00,2,6
4,1,2012/2013,2012-08-19 00:00:00+00:00,2,6
...,...,...,...,...,...
73,24558,2012/2013,2012-09-30 00:00:00+00:00,6,2
74,24558,2013/2014,2014-02-16 00:00:00+00:00,5,3
75,24558,2014/2015,2015-04-30 00:00:00+00:00,6,2
76,24558,2014/2015,2015-05-03 00:00:00+00:00,2,6


**Question** 17

Examine the highest total number of goals in each season, the overall highest total, and the highest total during July across all seasons.

In [21]:
SELECT
	season,
	MAX(home_goal + away_goal) AS max_goals,
	(SELECT MAX(home_goal + away_goal) FROM soccer.match) AS overall_max_goal,
	(SELECT MAX(home_goal + away_goal)
	 FROM soccer.match
	 WHERE id in 
		(SELECT id
		 FROM soccer.match
		 WHERE EXTRACT(MONTH from date) = 07
		)
	) AS july_max_goals
FROM soccer.match
GROUP BY season;

,season,max_goals,overall_max_goal,july_max_goals
0,2013/2014,10,11,7
1,2012/2013,11,11,7
2,2014/2015,10,11,7
3,2011/2012,10,11,7


In [22]:
-- The below code is also correct. It does NOT use Nested Subquery though

SELECT
	season,
	MAX(home_goal + away_goal) AS max_goals,
	(SELECT MAX(home_goal + away_goal) FROM soccer.match) AS overall_max_goal,
	(SELECT MAX(home_goal + away_goal)
	 FROM soccer.match
	 WHERE EXTRACT(MONTH from date) = 07
	) AS july_max_goals
FROM soccer.match
GROUP BY season;

,season,max_goals,overall_max_goal,july_max_goals
0,2013/2014,10,11,7
1,2012/2013,11,11,7
2,2014/2015,10,11,7
3,2011/2012,10,11,7


**Question** 18

Calculate the average number of goals per match in each country's league for the 2013/2014 season? Additionally, calculate the difference between each league's average goals and the overall season average.

In [23]:
SELECT
	l.name AS league,
	ROUND(AVG(m.home_goal + m.away_goal),2) AS avg_goals,
    -- Subtract the overall average from the league average
	ROUND(AVG(m.home_goal + m.away_goal) - 
		(SELECT AVG(home_goal + away_goal)
		 FROM soccer.match 
         WHERE season = '2013/2014'),2) AS diff
FROM soccer.league AS l
LEFT JOIN soccer.match AS m
ON l.country_id = m.country_id
-- Only include 2013/2014 results
WHERE season = '2013/2014'
GROUP BY l.name;

,league,avg_goals,diff
0,Switzerland Super League,2.89,0.12
1,Poland Ekstraklasa,2.64,-0.13
2,Netherlands Eredivisie,3.20,0.43
3,Scotland Premier League,2.75,-0.02
4,France Ligue 1,2.46,-0.31
5,Spain LIGA BBVA,2.75,-0.02
6,Germany 1. Bundesliga,3.16,0.39
7,Italy Serie A,2.72,-0.04
8,Portugal Liga ZON Sagres,2.37,-0.40
9,England Premier League,2.77,0.00


**Question** 19

Calculate and list the average number of matches per season where a team scored 5 or more goals? How does this differ by country?

In [24]:
SELECT
	c.name AS country,
    -- Calculate the average matches per season
	AVG(outer_s.matches) AS avg_seasonal_high_scores
FROM soccer.country AS c
-- Left join outer_s to country
LEFT JOIN (
	SELECT country_id, season,
    	COUNT(id) AS matches
	FROM (
    	-- List of matches where at least one team scored 5 or more goals.
		SELECT country_id, season, id
		FROM soccer.match
		WHERE home_goal >= 5 OR away_goal >= 5) AS inner_s
	GROUP BY country_id, season) AS outer_s
ON c.id = outer_s.country_id
GROUP BY country
ORDER BY avg_seasonal_high_scores DESC;

,country,avg_seasonal_high_scores
0,Spain,22.000000
1,Netherlands,21.000000
2,England,15.000000
3,Germany,13.750000
4,Belgium,11.333333
5,Italy,8.750000
6,Scotland,8.000000
7,France,8.000000
8,Portugal,7.500000
9,Switzerland,5.500000


In [25]:
-- Optimized the above query

SELECT
	name as country,
	AVG(sub.matches) AS avg_seasonal_high_scores
FROM soccer.country AS c
LEFT JOIN
	(SELECT
	 	country_id,
		season,
	 COUNT(id) AS matches
	 FROM soccer.match
	 WHERE
	 	home_goal >= 5
	 	OR away_goal >= 5
	 GROUP BY season, country_id
	 ORDER BY country_id, season
	) AS sub
ON c.id = sub.country_id
GROUP BY country
ORDER BY avg_seasonal_high_scores DESC;

,country,avg_seasonal_high_scores
0,Spain,22.000000
1,Netherlands,21.000000
2,England,15.000000
3,Germany,13.750000
4,Belgium,11.333333
5,Italy,8.750000
6,Scotland,8.000000
7,France,8.000000
8,Portugal,7.500000
9,Switzerland,5.500000


**Question** 20

What's the average number of goals scored in the month of August during each league in the season of 2013/2014.

In [26]:
WITH match_list AS (
    SELECT
  		country_id,
		(home_goal + away_goal) AS goals
    FROM soccer.match
    WHERE id IN (
    	SELECT id
		FROM soccer.match
		WHERE season = '2013/2014'
			AND EXTRACT(MONTH FROM date) = 08))

SELECT 
	l.name,
	AVG(match_list.goals) AS aug_average
FROM soccer.league AS l
LEFT JOIN match_list ON l.id = match_list.country_id
GROUP BY l.name
ORDER BY aug_average;

,name,aug_average
0,Switzerland Super League,1.937500
1,England Premier League,2.000000
2,France Ligue 1,2.027027
3,Scotland Premier League,2.137931
4,Poland Ekstraklasa,2.310345
5,Italy Serie A,2.750000
6,Spain LIGA BBVA,2.920000
7,Portugal Liga ZON Sagres,3.000000
8,Germany 1. Bundesliga,3.235294
9,Netherlands Eredivisie,3.414634


**Question** 21

Instead of displaying the team id's of the home team and away team in the match detail, get team name to replace the team id to make the output more readable.

Apply each of the following SQL techniques separately to solve the question three times. Use the knowledge of only one technique for each attempt:
- Simple Subqueries
- Correlated Queries
- Common Expression Tables

In [27]:
-- Get team names using concept of Simple Subquery

SELECT
	m.date,
    hometeam,
    awayteam,
    m.home_goal,
    m.away_goal
FROM soccer.match AS m

LEFT JOIN (
	SELECT match.id, team.team_long_name AS hometeam
	FROM soccer.match
	LEFT JOIN soccer.team
	-- Get the home team ID in the subquery
	ON match.hometeam_id = team.team_api_id) AS home
ON home.id = m.id

LEFT JOIN (
	SELECT match.id, team.team_long_name AS awayteam
	FROM soccer.match
	LEFT JOIN soccer.team
	-- Get the away team ID in the subquery
	ON match.awayteam_id = team.team_api_id) AS away
ON away.id = m.id;

,date,hometeam,awayteam,home_goal,away_goal
0,2011-07-29 00:00:00+00:00,Oud-Heverlee Leuven,RSC Anderlecht,2,1
1,2011-07-30 00:00:00+00:00,RAEC Mons,Standard de Liège,1,1
2,2011-07-30 00:00:00+00:00,KRC Genk,Beerschot AC,3,1
3,2011-07-30 00:00:00+00:00,KAA Gent,KSV Cercle Brugge,0,1
4,2011-07-30 00:00:00+00:00,Sporting Lokeren,SV Zulte-Waregem,0,0
...,...,...,...,...,...
12832,2014-09-23 00:00:00+00:00,FC Basel,FC Vaduz,3,1
12833,2014-09-23 00:00:00+00:00,FC Thun,FC Luzern,3,2
12834,2014-09-24 00:00:00+00:00,FC St. Gallen,Grasshopper Club Zürich,3,0
12835,2014-09-24 00:00:00+00:00,FC Zürich,FC Aarau,0,0


In [28]:
-- Get team names using concept of Correlated Subquery

SELECT
    m.date,
    (SELECT team_long_name
     FROM soccer.team AS t
     WHERE t.team_api_id = m.hometeam_id) AS hometeam,
    -- Connect the team to the match table
    (SELECT team_long_name
     FROM soccer.team AS t
     WHERE t.team_api_id = m.awayteam_id) AS awayteam,
    -- Select home and away goals
     m.home_goal,
     m.away_goal
FROM soccer.match AS m;

,date,hometeam,awayteam,home_goal,away_goal
0,2011-07-29 00:00:00+00:00,Oud-Heverlee Leuven,RSC Anderlecht,2,1
1,2011-07-30 00:00:00+00:00,RAEC Mons,Standard de Liège,1,1
2,2011-07-30 00:00:00+00:00,KRC Genk,Beerschot AC,3,1
3,2011-07-30 00:00:00+00:00,KAA Gent,KSV Cercle Brugge,0,1
4,2011-07-30 00:00:00+00:00,Sporting Lokeren,SV Zulte-Waregem,0,0
...,...,...,...,...,...
12832,2014-09-23 00:00:00+00:00,FC Basel,FC Vaduz,3,1
12833,2014-09-23 00:00:00+00:00,FC Thun,FC Luzern,3,2
12834,2014-09-24 00:00:00+00:00,FC St. Gallen,Grasshopper Club Zürich,3,0
12835,2014-09-24 00:00:00+00:00,FC Zürich,FC Aarau,0,0


In [29]:
-- Get team names using concept of Common Expression Table

WITH home AS (
  SELECT m.id, m.date, 
  		 t.team_long_name AS hometeam, m.home_goal
  FROM soccer.match AS m
  LEFT JOIN soccer.team AS t 
  ON m.hometeam_id = t.team_api_id),
-- Declare and set up the away CTE
away AS (
  SELECT m.id, m.date, 
  		 t.team_long_name AS awayteam, m.away_goal
  FROM soccer.match AS m
  LEFT JOIN soccer.team AS t 
  ON m.awayteam_id = t.team_api_id)
-- Select date, home_goal, and away_goal
SELECT 
	home.date,
    home.hometeam,
    away.awayteam,
    home.home_goal,
    away.away_goal
-- Join away and home on the id column
FROM home
INNER JOIN away
ON home.id = away.id;

,date,hometeam,awayteam,home_goal,away_goal
0,2011-07-29 00:00:00+00:00,Oud-Heverlee Leuven,RSC Anderlecht,2,1
1,2011-07-30 00:00:00+00:00,RAEC Mons,Standard de Liège,1,1
2,2011-07-30 00:00:00+00:00,KRC Genk,Beerschot AC,3,1
3,2011-07-30 00:00:00+00:00,KAA Gent,KSV Cercle Brugge,0,1
4,2011-07-30 00:00:00+00:00,Sporting Lokeren,SV Zulte-Waregem,0,0
...,...,...,...,...,...
12832,2014-09-23 00:00:00+00:00,FC Basel,FC Vaduz,3,1
12833,2014-09-23 00:00:00+00:00,FC Thun,FC Luzern,3,2
12834,2014-09-24 00:00:00+00:00,FC St. Gallen,Grasshopper Club Zürich,3,0
12835,2014-09-24 00:00:00+00:00,FC Zürich,FC Aarau,0,0


**Question** 22

Are there any teams in the database that have not played in their home country?

In [30]:
SELECT
	team_long_name,
	team_short_name
FROM soccer.team 
WHERE team_api_id NOT IN
    (SELECT DISTINCT hometeam_id FROM soccer.match);

,team_long_name,team_short_name
0,FCV Dender EH,DEN
1,KSV Roeselare,ROS
2,Tubize,TUB
3,Royal Excel Mouscron,MOU
4,KAS Eupen,EUP
5,Middlesbrough,MID
6,Portsmouth,POR
7,Birmingham City,BIR
8,Blackpool,BLA
9,Bournemouth,BOU


There are 52 teams without any home games recorded in this database. Let's analyze why it is so?

In [31]:
-- First, lets see which teams, if any, weren't part of any matches in any Eurpoean Leagues.

SELECT 
	team_long_name,
	team_short_name
FROM soccer.team 
WHERE team_api_id NOT IN
    (SELECT DISTINCT hometeam_id FROM soccer.match)
	OR team_api_id NOT IN
    (SELECT DISTINCT awayteam_id FROM soccer.match);

,team_long_name,team_short_name
0,FCV Dender EH,DEN
1,KSV Roeselare,ROS
2,Tubize,TUB
3,Royal Excel Mouscron,MOU
4,KAS Eupen,EUP
5,Middlesbrough,MID
6,Portsmouth,POR
7,Birmingham City,BIR
8,Blackpool,BLA
9,Bournemouth,BOU


Our first test itself revealed us the reason to why no home games were recorded for the above teams.
- The teams listed seem to be absent from participation in any matches within the European Leagues.
- Thus, we arrive at the highly probable conclusion that these teams primarily engage in local matches within their respective countries.

**Question** 23

Which teams were able to score 7 or more goals in their home country.

In [32]:
SELECT
	team_long_name,
	team_short_name
FROM soccer.team
WHERE team_api_id IN
	  (SELECT hometeam_ID 
       FROM soccer.match
       WHERE home_goal >= 7);

,team_long_name,team_short_name
0,Manchester United,MUN
1,Arsenal,ARS
2,Manchester City,MCI
3,Chelsea,CHE
4,Southampton,SOU
5,FC Bayern Munich,BMU
6,Juventus,JUV
7,Inter,INT
8,Roda JC Kerkrade,ROD
9,PSV,PSV


**Question** 24

Let's identify whether a match in Germany included FC Bayern Munich, FC Schalke 04, or neither as the home team.

In [33]:
-- Let's get the team_ids of the desired teams.

SELECT
	team_long_name,
	team_api_id
FROM soccer.team
WHERE team_long_name IN ('FC Schalke 04', 'FC Bayern Munich');

,team_long_name,team_api_id
0,FC Bayern Munich,9823
1,FC Schalke 04,10189


In [34]:
-- Let's get the country_id for Germany.

SELECT *
FROM soccer.country
WHERE soccer.country.name = 'Germany';

,id,name
0,7809,Germany


In [35]:
-- Main Query

SELECT 
	CASE WHEN hometeam_id = 10189 THEN 'FC Schalke 04'
        WHEN hometeam_id = 9823 THEN 'FC Bayern Munich'
        ELSE 'Other' END AS home_team,
	COUNT(id) AS total_matches
FROM soccer.match
WHERE match.country_id = 7809
GROUP BY home_team;

,home_team,total_matches
0,Other,1088
1,FC Schalke 04,68
2,FC Bayern Munich,68


**Question** 25

Create a list of matches in the 2011/2012 season where Barcelona was the home team. Also identify the opponent and the winner of the match.

In [36]:
SELECT 
	m.date,
	t.team_long_name AS opponent,
	CASE WHEN m.home_goal > m.away_goal THEN 'Barcelona win!'
        WHEN m.home_goal < m.away_goal THEN 'Barcelona loss :(' 
        ELSE 'Tie' END AS outcome
FROM soccer.match AS m
LEFT JOIN soccer.team AS t 
ON m.awayteam_id = t.team_api_id
WHERE m.hometeam_id = 8634;

,date,opponent,outcome
0,2011-10-29 00:00:00+00:00,RCD Mallorca,Barcelona win!
1,2011-11-19 00:00:00+00:00,Real Zaragoza,Barcelona win!
2,2011-12-03 00:00:00+00:00,Levante UD,Barcelona win!
3,2011-11-29 00:00:00+00:00,Rayo Vallecano,Barcelona win!
4,2012-01-15 00:00:00+00:00,Real Betis Balompié,Barcelona win!
...,...,...,...
71,2015-04-28 00:00:00+00:00,Getafe CF,Barcelona win!
72,2015-05-09 00:00:00+00:00,Real Sociedad,Barcelona win!
73,2015-05-23 00:00:00+00:00,RC Deportivo de La Coruña,Tie
74,2014-09-27 00:00:00+00:00,Granada CF,Barcelona win!


**Question** 26

Similar to the previous question, determine the outcome of Barcelona's matches where they played as the away team.

In [37]:
SELECT  
	m.date,
	t.team_long_name AS opponent,
	CASE WHEN m.home_goal < m.away_goal THEN 'Barcelona win!'
        WHEN m.home_goal > m.away_goal THEN 'Barcelona loss :(' 
        ELSE 'Tie' END AS outcome
FROM soccer.match AS m
LEFT JOIN soccer.team AS t 
ON m.hometeam_id = t.team_api_id
WHERE m.awayteam_id = 8634;

,date,opponent,outcome
0,2012-01-22 00:00:00+00:00,Málaga CF,Barcelona win!
1,2011-10-25 00:00:00+00:00,Granada CF,Barcelona win!
2,2011-11-06 00:00:00+00:00,Athletic Club de Bilbao,Tie
3,2011-11-26 00:00:00+00:00,Getafe CF,Barcelona loss :(
4,2011-12-10 00:00:00+00:00,Real Madrid CF,Barcelona win!
...,...,...,...
71,2015-05-17 00:00:00+00:00,Atlético Madrid,Barcelona win!
72,2014-09-21 00:00:00+00:00,Levante UD,Barcelona win!
73,2014-09-24 00:00:00+00:00,Málaga CF,Tie
74,2014-10-04 00:00:00+00:00,Rayo Vallecano,Barcelona win!


**Question** 27

Barcelona and Real Madrid have been rival teams for more than 80 years. Matches between these two teams are given the name El Clásico (The Classic). So, let's query a list of matches played between these two rivals and inspect the outcome of the match.

NOTE : Barcelona (id = 8634) || Real Madrid (id = 8633)

In [38]:
SELECT
	date,
	CASE WHEN hometeam_id = 8634 THEN 'FC Barcelona' 
         ELSE 'Real Madrid CF' END as home,
	CASE WHEN awayteam_id = 8634 THEN 'FC Barcelona' 
         ELSE 'Real Madrid CF' END as away,
	-- Identify all possible match outcomes
	CASE WHEN home_goal > away_goal AND hometeam_id = 8634 THEN 'Barcelona win!'
         WHEN home_goal > away_goal AND hometeam_id = 8633 THEN 'Real Madrid win!'
         WHEN home_goal < away_goal AND awayteam_id = 8634 THEN 'Barcelona win!'
         WHEN home_goal < away_goal AND awayteam_id = 8633 THEN 'Real Madrid win!'
         ELSE 'Tie!' END AS outcome
FROM soccer.match
WHERE (awayteam_id = 8634 OR hometeam_id = 8634)
      AND (awayteam_id = 8633 OR hometeam_id = 8633);

,date,home,away,outcome
0,2011-12-10 00:00:00+00:00,Real Madrid CF,FC Barcelona,Barcelona win!
1,2012-04-21 00:00:00+00:00,FC Barcelona,Real Madrid CF,Real Madrid win!
2,2013-03-02 00:00:00+00:00,Real Madrid CF,FC Barcelona,Real Madrid win!
3,2012-10-07 00:00:00+00:00,FC Barcelona,Real Madrid CF,Tie!
4,2013-10-26 00:00:00+00:00,FC Barcelona,Real Madrid CF,Barcelona win!
5,2014-03-23 00:00:00+00:00,Real Madrid CF,FC Barcelona,Barcelona win!
6,2015-03-22 00:00:00+00:00,FC Barcelona,Real Madrid CF,Barcelona win!
7,2014-10-25 00:00:00+00:00,Real Madrid CF,FC Barcelona,Real Madrid win!


**Question** 28

Create a data set of games played by Legia Warszawa (Warsaw League), the top ranked team in Poland. Compare their individual game performance to the overall average for that season.

Legia Warszawa ID = 8673

In [39]:
SELECT
	date,
	season,
	home_goal,
	away_goal,
	CASE WHEN hometeam_id = 8673 THEN 'home' 
		 ELSE 'away' END AS warsaw_location,
    -- Calculate the average goals scored partitioned by season
    AVG(home_goal) OVER(PARTITION BY season) AS season_homeavg,
    AVG(away_goal) OVER(PARTITION BY season) AS season_awayavg
FROM soccer.match
-- Filter the data set for Legia Warszawa matches only
WHERE 
	hometeam_id = 8673 
    OR awayteam_id = 8673
ORDER BY (home_goal + away_goal) DESC;

,date,season,home_goal,away_goal,warsaw_location,season_homeavg,season_awayavg
0,2013-09-14 00:00:00+00:00,2013/2014,3,5,away,1.766667,1.233333
1,2014-09-13 00:00:00+00:00,2014/2015,4,3,home,1.566667,1.333333
2,2013-07-20 00:00:00+00:00,2013/2014,5,1,home,1.766667,1.233333
3,2013-10-20 00:00:00+00:00,2013/2014,4,1,home,1.766667,1.233333
4,2013-06-02 00:00:00+00:00,2012/2013,5,0,home,1.566667,1.133333
...,...,...,...,...,...,...,...
115,2013-05-30 00:00:00+00:00,2012/2013,0,0,away,1.566667,1.133333
116,2013-04-27 00:00:00+00:00,2012/2013,0,0,away,1.566667,1.133333
117,2013-03-02 00:00:00+00:00,2012/2013,0,0,home,1.566667,1.133333
118,2015-04-24 00:00:00+00:00,2014/2015,0,0,away,1.566667,1.333333


**Question** 29

Calculate the running average of home and away goals scored Legia Warszawa, and their opponents, partitioned by the month in each season.

In [40]:
SELECT 
	date,
	season,
	home_goal,
	away_goal,
	CASE WHEN hometeam_id = 8673 THEN 'home' 
         ELSE 'away' END AS warsaw_location,
    AVG(home_goal) OVER(PARTITION BY season, 
         	EXTRACT(MONTH FROM date)) AS season_mo_home,
    AVG(away_goal) OVER(PARTITION BY season,
            EXTRACT(MONTH FROM date)) AS season_mo_away
FROM soccer.match
WHERE 
	hometeam_id = 8673
    OR awayteam_id = 8673
ORDER BY (home_goal + away_goal) DESC;

,date,season,home_goal,away_goal,warsaw_location,season_mo_home,season_mo_away
0,2013-09-14 00:00:00+00:00,2013/2014,3,5,away,2.250000,2.500000
1,2014-09-13 00:00:00+00:00,2014/2015,4,3,home,2.000000,2.666667
2,2013-07-20 00:00:00+00:00,2013/2014,5,1,home,2.500000,2.000000
3,2014-08-09 00:00:00+00:00,2014/2015,5,0,home,2.000000,1.000000
4,2012-10-28 00:00:00+00:00,2012/2013,3,2,home,1.666667,2.000000
...,...,...,...,...,...,...,...
115,2011-10-30 00:00:00+00:00,2011/2012,0,0,away,1.000000,0.250000
116,2011-11-07 00:00:00+00:00,2011/2012,0,0,away,1.750000,0.000000
117,2015-02-22 00:00:00+00:00,2014/2015,0,0,away,0.500000,1.500000
118,2012-03-30 00:00:00+00:00,2011/2012,0,0,away,0.600000,0.400000


**Question** 30

Calculate the running total of goals scored by the FC Utrecht when they were the home team during the 2011/2012 season.

FC Utrecht = 9908

In [41]:
SELECT 
	date,
	home_goal,
	away_goal,
    -- Create a running total and running average of home goals
    SUM(home_goal) OVER(ORDER BY date 
         ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS running_total,
    AVG(home_goal) OVER(ORDER BY date 
         ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS running_avg
FROM soccer.match
WHERE 
	hometeam_id = 9908 
	AND season = '2011/2012';

,date,home_goal,away_goal,running_total,running_avg
0,2011-08-14 00:00:00+00:00,2,2,2,2.000000
1,2011-08-27 00:00:00+00:00,3,1,5,2.500000
2,2011-09-18 00:00:00+00:00,2,2,7,2.333333
3,2011-10-01 00:00:00+00:00,3,0,10,2.500000
4,2011-10-22 00:00:00+00:00,1,4,11,2.200000
5,2011-11-06 00:00:00+00:00,6,4,17,2.833333
6,2011-12-04 00:00:00+00:00,2,6,19,2.714286
7,2011-12-11 00:00:00+00:00,2,2,21,2.625000
8,2012-01-22 00:00:00+00:00,1,1,22,2.444444
9,2012-02-12 00:00:00+00:00,1,1,23,2.300000


**Question** 31

Do FC Utrecht score more goals at the end of the season as the home or away team?

In [42]:
-- Running total of goals scored by the FC Utrecht as away team

SELECT 
     date,
     home_goal,
     away_goal,
     -- Create a running total and running average of home goals
     SUM(home_goal) OVER(ORDER BY date DESC
         ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING) AS running_total,
     AVG(home_goal) OVER(ORDER BY date DESC
         ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING) AS running_avg
FROM soccer.match
WHERE 
     awayteam_id = 9908 
     AND season = '2011/2012';

,date,home_goal,away_goal,running_total,running_avg
0,2012-05-06 00:00:00+00:00,1,3,25,1.470588
1,2012-04-21 00:00:00+00:00,0,2,24,1.500000
2,2012-04-12 00:00:00+00:00,3,0,24,1.600000
3,2012-03-25 00:00:00+00:00,3,1,21,1.500000
4,2012-03-11 00:00:00+00:00,1,1,18,1.384615
5,2012-02-26 00:00:00+00:00,1,0,17,1.416667
6,2012-02-05 00:00:00+00:00,0,2,16,1.454545
7,2012-01-28 00:00:00+00:00,2,0,16,1.600000
8,2011-12-17 00:00:00+00:00,1,0,14,1.555556
9,2011-11-25 00:00:00+00:00,2,0,13,1.625000


Comparing the above results, it is evident that FC Utrecht scores as the home team.

**Question** 32

Who defeated Manchester United in the 2013/2014 season? Rank the matches by the number of goals they lost the match.

In [43]:
-- Set up the home team CTE
WITH home AS (
  SELECT m.id, t.team_long_name,
	  CASE WHEN m.home_goal > m.away_goal THEN 'MU Win'
		   WHEN m.home_goal < m.away_goal THEN 'MU Loss' 
  		   ELSE 'Tie' END AS outcome
  FROM soccer.match AS m
  LEFT JOIN soccer.team AS t ON m.hometeam_id = t.team_api_id),
-- Set up the away team CTE
away AS (
  SELECT m.id, t.team_long_name,
	  CASE WHEN m.home_goal > m.away_goal THEN 'MU Loss'
		   WHEN m.home_goal < m.away_goal THEN 'MU Win' 
  		   ELSE 'Tie' END AS outcome
  FROM soccer.match AS m
  LEFT JOIN soccer.team AS t ON m.awayteam_id = t.team_api_id)

-- Select columns and and rank the matches by goal difference
SELECT DISTINCT
    date,
    home.team_long_name AS home_team,
    away.team_long_name AS away_team,
    m.home_goal, m.away_goal,
    RANK() OVER(ORDER BY ABS(home_goal - away_goal) DESC) as match_rank
-- Join the CTEs onto the match table
FROM soccer.match AS m
LEFT JOIN home ON m.id = home.id
LEFT JOIN away ON m.id = away.id
WHERE m.season = '2014/2015'
      AND ((home.team_long_name = 'Manchester United' AND home.outcome = 'MU Loss')
      OR (away.team_long_name = 'Manchester United' AND away.outcome = 'MU Loss'))
ORDER BY match_rank, date;

,date,home_team,away_team,home_goal,away_goal,match_rank
0,2015-04-26 00:00:00+00:00,Everton,Manchester United,3,0,1
1,2014-09-21 00:00:00+00:00,Leicester City,Manchester United,5,3,2
2,2014-08-16 00:00:00+00:00,Manchester United,Swansea City,1,2,3
3,2014-11-02 00:00:00+00:00,Manchester City,Manchester United,1,0,3
4,2015-01-11 00:00:00+00:00,Manchester United,Southampton,0,1,3
5,2015-02-21 00:00:00+00:00,Swansea City,Manchester United,2,1,3
6,2015-04-18 00:00:00+00:00,Chelsea,Manchester United,1,0,3
7,2015-05-02 00:00:00+00:00,Manchester United,West Bromwich Albion,0,1,3


**Question** 33

In a soccer league, points are assigned to teams based on the result of a game. Here, let's assume that 3 points are awarded for a win, 1 for a tie, and 0 for a defeat. Using the above assumption, let's calculate the running total of points earned by the team "Chelsea" (team id 8455) in the season "2014/2015".

In [44]:
WITH results AS (
	SELECT date, hometeam_id, awayteam_id, home_goal, away_goal,
		CASE WHEN hometeam_id = 8455 AND home_goal > away_goal THEN 3
			WHEN hometeam_id = 8455 AND home_goal = away_goal THEN 1
			WHEN awayteam_id = 8455 AND home_goal < away_goal THEN 3
			WHEN awayteam_id = 8455 AND home_goal = away_goal THEN 1
			ELSE 0 END AS points
	FROM soccer.match
	WHERE (hometeam_id = 8455 OR awayteam_id = 8455)
		AND season = '2014/2015'
)

SELECT date, points,
	SUM(points) OVER(ORDER BY date ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as rolling_point
FROM results;

,date,points,rolling_point
0,2014-08-18 00:00:00+00:00,3,3
1,2014-08-23 00:00:00+00:00,3,6
2,2014-08-30 00:00:00+00:00,3,9
3,2014-09-13 00:00:00+00:00,3,12
4,2014-09-21 00:00:00+00:00,1,13
5,2014-09-27 00:00:00+00:00,3,16
6,2014-10-05 00:00:00+00:00,3,19
7,2014-10-18 00:00:00+00:00,3,22
8,2014-10-26 00:00:00+00:00,1,23
9,2014-11-01 00:00:00+00:00,3,26
